# transform raw FCE data into CSV
* no need to modify this file unless we need to change the database structure

### Manually checked documents:
* doc2.xml 
* doc865.xml (did not check essays)

In [147]:
import xml.etree.ElementTree as ET
import re
import os
import pandas as pd
import numpy as np

In [149]:
directory = "raw_data/fce-released-dataset/dataset/"

In [244]:
"""
Assumptions: 
Everyone uses punctuation with correct spacing. (i.e. no white space in front of a punctuation.)
Spacing in front of each paragraph is ignored. (i.e. all sentences are left indented.)
Every document has similar XML structures.
"""
def get_coded_answer(paragraphs, debug=False):
    result = []
    mistakes = []
    for p in paragraphs:
        sentence = [p.text.strip() + ' '] if p.text else []
        for ns in p:
            mistakes.append(ns.attrib.get('type'))
            for child in ns:
                all_incorrect_text = []
                if child.tag == 'i' and child.text is not None:
                    incorrect_text = child.text.strip();
                    all_incorrect_text.append(incorrect_text + ' ')
                    current = child
                    while current:
                        current = current.find('./NS/i')
                        if current is not None and current.text is not None:
                            incorrect_text = current.text.strip()
                            all_incorrect_text.append(incorrect_text + ' ')
                    sentence.append(''.join(all_incorrect_text) + '')
            if ns.tail:
                p_tail = ns.tail.strip();
                sentence.append(p_tail + ' ')
        sentence = ''.join(sentence)
        sentence = ' '.join([word.strip() for word in sentence.split()])
        sentence = re.sub(r'\s+([.,!?;:])', r'\1', sentence)
        result.append(sentence)
        if debug:
            print("__________________________________________________________________________")
            print(sentence)
    return '\n'.join(result), mistakes

In [245]:
"""
add 1 XML file to df
"""
def add_XML(dir, df, debug=False):
    student = {"ID": '/'.join(dir.split("/")[-2:])}
    # Load the XML file
    tree = ET.parse(dir) 
    root = tree.getroot()
    
    # Candidate information
    candidate = root.find('.//candidate')
    student['native_language'] = candidate.find('./personnel/language').text
    student['age'] = candidate.find('./personnel/age').text if candidate.find('./personnel/age') is not None else np.nan
    student['total_score'] = candidate.find('./score').text
    
    
    # Test Answer 1
    new_row = student.copy()
    answer = root.find('.//text/answer1')
    new_row["question_number"] = answer.find('./question_number').text
    new_row["exam_score"] = answer.find('./exam_score').text if answer.find('./exam_score') is not None else np.nan
    new_row["answer"], new_row["mistakes_tag"] = get_coded_answer(answer.find('./coded_answer'), debug=False)
    df.loc[len(df)] = new_row
    
    # debug
    if debug:
        print(f'Native Language: {student["native_language"]}, Age: {student["age"]}, Total Score: {student["total_score"]}')
        print(f'----Question Number: {new_row["question_number"]}, Exam Score: {new_row["exam_score"]}')
        print(f'----Coded Answer: \n{new_row["answer"]}')
    
    
    # Test Answer 2
    new_row = student.copy()
    answer = root.find('.//text/answer2')
    if answer is not None:
        new_row["question_number"] = answer.find('./question_number').text
        new_row["exam_score"] = answer.find('./exam_score').text if answer.find('./exam_score') is not None else np.nan
        new_row["answer"], new_row["mistakes_tag"] = get_coded_answer(answer.find('./coded_answer'), debug=False)
        df.loc[len(df)] = new_row
        
        # debug
        if debug:
            print(f'----Question Number: {new_row["question_number"]}, Exam Score: {new_row["exam_score"]}')
            print(f'----Coded Answer: \n{new_row["answer"]}')
    
    return df

In [238]:
"""
sort directories in custom order
"""
def sort_key(file_path):
    match = re.search(r'/(\d+)_(\d+)_(\d+)/doc(\d+)\.xml$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3)), int(match.group(4))
    return float('inf'), float('inf'), float('inf'), float('inf')

In [246]:
"""
Example: doc2.xml
"""
dir = f'{directory}/0100_2000_6/doc2.xml'
df = pd.DataFrame(columns=['ID', 'native_language','age', 'total_score', 'question_number', 'exam_score', 'answer', 'mistakes_tag'])

add_XML(dir, df, debug=True)

Native Language: Chinese, Age: 16-20, Total Score: 24.0
----Question Number: 1, Exam Score: 3.2
----Coded Answer: 
Dear Mr Ryan.
Thanks for you letter. I am so exciting that I have won the first prize. I will give you all information you need and ask some questions.
I could only travel on July. As you know, I am a student and the nearest holiday is summer holiday. But I have booked a flight to home at the beginning of Auguest. And also I would like to go on summer.
The accommodation I would like to live in tents. Because I never live in tents before. I think it is great and I want to try it.
I like doing sports. I would like to play basketball and golf when I am at the Camp. I play basketball a lot and I am a member of our college term. But I am not very good at golf.
And also I want to ask some questions. What clothes should I taken? How much money should I taken? And how could we meet at the airport? I am looking forward your reply.
Yours sincerely.
----Question Number: 2, Exam Score

,ID,native_language,age,total_score,question_number,exam_score,answer,mistakes_tag
0,0100_2000_6/doc2.xml,Chinese,16-20,24.0,1,3.2,Dear Mr Ryan.\nThanks for you letter. I am so ...,"[RP, DD, RJ, MD, TV, RT, MD, UT, S, RT, MT, MP..."
1,0100_2000_6/doc2.xml,Chinese,16-20,24.0,2,2.3,As our class is going to mark a short video ab...,"[RV, TV, UQ, MD, RP, MT, FN, MT, FV, RT, S, MD..."


In [230]:
"""
sort directories into the given order
"""
directories = []
for subdir, dirs, files in os.walk(directory):
    for file in files:
        directories.append(os.path.join(subdir, file))

directories = sorted(directories, key=sort_key)
print(directories)


['raw_data/fce-released-dataset/dataset/0100_2000_6/doc2.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc4.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc6.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc11.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc15.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc24.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc27.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc34.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc41.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc48.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc49.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc50.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc57.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc58.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc59.xml', 'raw_data/fce-released-dataset/dataset/0100_2000_6/doc61.

In [248]:
"""
OMG I DID IT
"""
df = pd.DataFrame(columns=['ID', 'native_language','age', 'total_score', 'question_number', 'exam_score', 'answer', 'mistakes_tag'])

for dir in directories:
    df = add_XML(dir, df)

df

,ID,native_language,age,total_score,question_number,exam_score,answer,mistakes_tag
0,0100_2000_6/doc2.xml,Chinese,16-20,24.0,1,3.2,Dear Mr Ryan.\nThanks for you letter. I am so ...,"[RP, DD, RJ, MD, TV, RT, MD, UT, S, RT, MT, MP..."
1,0100_2000_6/doc2.xml,Chinese,16-20,24.0,2,2.3,As our class is going to mark a short video ab...,"[RV, TV, UQ, MD, RP, MT, FN, MT, FV, RT, S, MD..."
2,0100_2000_6/doc4.xml,Chinese,21-25,17.0,1,2.2,"Dear Madam Helen Ryan,\nI have received your l...","[RN, TV, RY, AGV, MA, W, R, MT, RC, RA, MD, RP..."
3,0100_2000_6/doc4.xml,Chinese,21-25,17.0,3,2.1,I agree! Sometimes shopping is not always enjo...,"[UT, U, RP, MP, L, RT, UT, FV, R, RV, R, RN, T..."
4,0100_2000_6/doc6.xml,Polish,16-20,26.0,1,3.1,Dir Madam\nI'm writing to you in reply to your...,"[S, S, S, RT, SX, RP, S, MT, FV, RD, S, S, RT,..."
...,...,...,...,...,...,...,...,...
2476,0102_2001_6/doc3139.xml,Japanese,26-30,36.00,3,4.3,"I had never taken birthdays serious, both my o...","[DY, RQ, RC, MA, FD, RV, AGA, S, FN, L, RT]"
2477,0102_2001_12/doc2871.xml,Polish,26-30,40.00,1,5.3,"Dear Mary,\nI'm really happy to hear from you!...","[TV, RP, MD, RC, RP]"
2478,0102_2001_12/doc2871.xml,Polish,26-30,40.00,3,5.2,It was three o'clock in the morning when the p...,"[RD, TV, FV, RD, MY, TV, TV]"
2479,0102_2001_12/doc2894.xml,Russian,16-20,40.00,1,5.3,Dear John!\nI was very happy to receive your l...,"[UP, RP, RJ, RP, S, R, TV, RD, MD, MD, RP, RP,..."


In [249]:
df[df.isna().any(axis=1)]

,ID,native_language,age,total_score,question_number,exam_score,answer,mistakes_tag
308,0100_2000_6/doc865.xml,French,NaN,25.0,1,2.3,"Misses Helen Ryan,\nFirst of all I would like ...","[MJ, S, S, S, R, RD, U, FV, S, R, RA, RT, MP, ..."
309,0100_2000_6/doc865.xml,French,NaN,25.0,3,3.2,Shopping is not always enjoyable.\nHowever it ...,"[RC, RP, ID, UY, L, RV, AGA, RN, MP, RD, RA, D..."
390,0100_2000_6/doc1153.xml,Spanish,NaN,24.0,1,3.1,"Hi Helen,\nIm very excited because I won the f...","[MP, RP, MP, RT, RP, R, S, RT, MP, DY, SX, MP,..."
391,0100_2000_6/doc1153.xml,Spanish,NaN,24.0,4,3.1,Dear Kim Im having a great time here in brazil...,"[MP, RP, R, MP, UP, MD, L, MP, RP, R, SA, FD, ..."
392,0100_2000_6/doc1157.xml,Korean,NaN,32.0,1,4.2,"Dear Sir/Madam,\nI am writing in order to give...","[RV, UD, W, IV, RT, MT, M, RT, MD, RN, RT]"
393,0100_2000_6/doc1157.xml,Korean,NaN,32.0,4,4.1,"Dear Kim,\nHow are you? Thanks for your last l...","[MY, MT, FV, IV, ID, SX]"
997,0100_2000_12/doc548.xml,Japanese,26-30,27.00,2,NaN,Some people said that famous people such as po...,"[TV, AGV, RV, MP, MD, S, TV, TV, MD, TV, RV, U..."
1111,0100_2000_12/doc1581.xml,Turkish,21-25,25.00,3,NaN,Futuristic Designs Of People's Homes\nTechnolo...,"[W, DJ, IN, CN, MP, AS, MP, AGN, RV, UQ, CN, F..."
1162,0100_2000_12/doc2302.xml,German,16-20,37.00,5b,NaN,Dear Jack\nThanks a lot for your letter. I am ...,"[RP, AS, RP, L, RP, RN, RV, MP, S, MQ]"
1281,0100_2001_6/doc3063.xml,German,NaN,26.00,1,3.3,Dear Mrs. Smith\nThank you very much for your ...,"[UP, UP, MC, RV, RT, W, RT, S, MP, MD, RP, MP,..."


In [250]:
df.to_csv(f"data/fce.csv", index=False)